In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
!ls 'Videos7439' | wc -l

7439


In [3]:
!ls 'Spectrograms7439' | wc -l

7439


In [4]:
videos_folder = 'Videos7439'
spectrograms_folder = 'Spectrograms7439'

In [35]:
import cv2
import os
import sys
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image
from facenet_pytorch import MTCNN
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
import face_alignment
#import dlib
import requests
import csv
import itertools


In [36]:

class GoogLeNetModified(models.GoogLeNet):
    def _transform_input(self, x):
        if x.size(1) == 1:
            # If the input has only one channel, replicate it to create three channels
            x = x.expand(-1, 3, -1, -1)

        x_ch0 = torch.unsqueeze(x[:, 0], 1) * (0.229 / 0.5) + (0.485 - 0.5) / 0.5
        x_ch1 = torch.unsqueeze(x[:, 1], 1) * (0.224 / 0.5) + (0.456 - 0.5) / 0.5
        x_ch2 = torch.unsqueeze(x[:, 2], 1) * (0.225 / 0.5) + (0.406 - 0.5) / 0.5
        x = torch.cat((x_ch0, x_ch1, x_ch2), 1)

        return x
        
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.features = GoogLeNetModified()
        # Modify the first layer to accept 3 channel input (for RGB images)
        self.features.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        # Modify the final layer to output the desired feature size
        self.features.fc = nn.Linear(self.features.fc.in_features, num_classes)

    def forward(self, x):
        x = self.features(x)

        # Assuming the output is a tensor inside the GoogLeNetOutputs object
        logits_tensor = x.logits if hasattr(x, 'logits') else x  # Adjust accordingly based on the structure

        # Apply softmax directly on the logits tensor
        x_softmax = torch.nn.functional.softmax(logits_tensor, dim=1)
        return x_softmax






In [37]:
def extract_frame(video_path):
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    mid_frame_index = frame_count // 2  # Index of the frame in the middle of the video
    cap.set(cv2.CAP_PROP_POS_FRAMES, mid_frame_index)
    ret, frame = cap.read()
    if ret:
        cap.release()
        return frame
    else:
        cap.release()
        return None

In [38]:
def detect_face(frame):
    mtcnn = MTCNN()
    boxes, _ = mtcnn.detect(frame)
    if boxes is not None:
        # Assuming only one face in the frame
        box = boxes[0]
        x1, y1, x2, y2 = box
        # Crop the frame to the detected face
        cropped_frame = frame[int(y1):int(y2), int(x1):int(x2)]
        return cropped_frame
    else:
        return None

In [39]:
import cv2
import face_alignment
import requests
import os

# Function to download the pretrained face alignment model if it doesn't exist
def download_face_alignment_model(url, save_path):
    if not os.path.exists(save_path):
        print("Downloading pretrained face alignment model...")
        response = requests.get(url)
        with open(save_path, 'wb') as f:
            f.write(response.content)
        print("Download complete.")

# Specify the URL of the pretrained face alignment model
face_alignment_model_url = "https://github.com/1adrianb/face-alignment-models/releases/download/2.0.1/2DFAN4-11f355bf06.pth.tar"

# Download the pretrained face alignment model if it doesn't exist
face_alignment_model_path = os.path.abspath("2DFAN4-11f355bf06.pth.tar")
download_face_alignment_model(face_alignment_model_url, face_alignment_model_path)

# Initialize face alignment model
fa = face_alignment.FaceAlignment(2, flip_input=False,device='cpu')  # 2 corresponds to 2D landmarks

def align_face(frame):
    # Perform face alignment
    aligned_faces = fa.get_landmarks(frame)
    if aligned_faces is not None:
        aligned_face = aligned_faces[0]  # Assuming only one face in the frame
        return aligned_face
    else:
        return None


In [40]:
def preprocess_image(frame):
    frame_pil = Image.fromarray(frame.astype('uint8'))
    frame_pil = frame_pil.convert('RGB')  # Convert to RGB format
    frame_np = np.array(frame_pil)
    print(frame_np.shape)
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize using ImageNet mean and std
    ])
    frame_tensor = transform(frame_pil)
    return frame_tensor


In [41]:
def preprocess_spectrogram(image_path):
    img = Image.open(image_path).convert('RGB')  # Convert to grayscale
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize to match ViT input size
        transforms.ToTensor(),           # Convert to tensor
    ])
    img_tensor = transform(img)
    return img_tensor

In [42]:
def load_dataset(videos_folder):
    X = []
    y = []
    skipped_files = []
    video_files = [file for file in sorted(os.listdir(videos_folder)) if file.endswith(".flv")]
    for video_file in tqdm(video_files):
        video_path = os.path.join(videos_folder, video_file)
        frame = extract_frame(video_path)
        if frame is not None:
            cropped_face = detect_face(frame)
            if cropped_face is not None:
                preprocessed_face = preprocess_image(cropped_face)
                X.append(preprocessed_face)
                label = video_file.split("_")[2].split(".")[0]  # Adjusted to handle different file extensions
                if label == "HAP":
                    y.append(0)
                elif label == "SAD":
                    y.append(1)
                elif label == "ANG":
                    y.append(2)
                elif label == "DIS":
                    y.append(3)
                elif label == "FEA":
                    y.append(4)
                elif label == "NEU":
                    y.append(5)
            else:
                print(f"No face detected in {video_file}. Skipping.")
                skipped_files.append(video_file[:-3])
        else:
            print(f"Failed to extract frame from {video_file}. Skipping.")
            skipped_files.append(video_file[:-3])
    return X, y, skipped_files

In [43]:
def load_spectrogram_dataset(spectrograms_folder, skipped_files):
    X = []
    y = []
    # List all files in the input folder
    files = sorted(os.listdir(spectrograms_folder))
    # Iterate over files in the folder
    for filename in tqdm(files):
        if filename.endswith(".png") and filename[:-3] not in skipped_files:  # Assuming mel spectrograms are stored as PNG files
            input_path = os.path.join(spectrograms_folder, filename)
            img_tensor = preprocess_spectrogram(input_path)
            X.append(img_tensor)
            # Extract label from filename (assuming filename is in format "abc_IEO_label_xyz.png")
            label = filename.split("_")[2]
            if label == "HAP":
                y.append(0)
            elif label == "SAD":
                y.append(1)
            elif label == "ANG":
                y.append(2)
            elif label == "DIS":
                y.append(3)
            elif label == "FEA":
                y.append(4)
            elif label == "NEU":
                y.append(5)
    return X, y

In [44]:
def train_model(model, criterion, optimizer, train_loader, device):
    model.train()
    running_loss = 0.0
    correct_preds = 0
    total_preds = 0
    for inputs, labels in tqdm(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        correct_preds += (predicted == labels).sum().item()
        total_preds += labels.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    accuracy = correct_preds / total_preds
    return epoch_loss, accuracy

In [45]:
def test_model(model, criterion, test_loader, device):
    model.eval()
    running_loss = 0.0
    correct_preds = 0
    total_preds = 0
    with torch.no_grad():
        for inputs, labels in tqdm(test_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            correct_preds += (predicted == labels).sum().item()
            total_preds += labels.size(0)
    epoch_loss = running_loss / len(test_loader.dataset)
    accuracy = correct_preds / total_preds
    return epoch_loss, accuracy

In [46]:
# def extract_features_from_folder(input_folder):
#     # Initialize the model
#     model = CNN(num_classes=3)  # 3 classes for HAPPY, SAD, ANGRY
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     model.to(device)
#     model.eval()  # Set the model to evaluation mode

#     # List all files in the input folder
#     files = os.listdir(input_folder)

#     # Iterate over files in the folder
#     for filename in files:
#         if filename.endswith(".png"):  # Assuming mel spectrograms are stored as PNG files
#             input_path = os.path.join(input_folder, filename)
#             img_tensor = preprocess_image(input_path)
#             img_tensor = img_tensor.to(device)
#             with torch.no_grad():
#                 output_features = model(img_tensor)
#             print(f"Features extracted for {filename}: {output_features}")


In [47]:
# extract_features_from_folder('/content/drive/MyDrive/csci535/melspec')

In [48]:
# !python3 melspec_to_features_cnn.py /content/drive/MyDrive/csci535/melspec


In [49]:
# if __name__ == "__main__":
#     # Check if input arguments are provided
#     # if len(sys.argv) != 2:
#     #     print("Usage: python melspec_to_features_cnn.py input_folder")
#     #     sys.exit(1)

#     # input_folder = sys.argv[1]
#     input_folder = '/content/drive/MyDrive/csci535/melspec'
#     # Check if input folder exists
#     if not os.path.exists(input_folder):
#         print("Input folder does not exist.")
#         sys.exit(1)

#     # Load dataset and split into train and test sets
#     X, y = load_dataset(input_folder)
#     print(f"Total number of samples: {len(X)}")
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
#     print(f"Number of train samples: {len(X_train)}", f"Number of test samples: {len(X_test)}")
#     # Initialize the model
#     model = CNN(num_classes=3)  # 3 classes for HAPPY, SAD, ANGRY
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     model.to(device)

#     # Define loss function and optimizer
#     _lr = 0.001
#     criterion = nn.CrossEntropyLoss()
#     optimizer = optim.Adam(model.parameters(), lr=_lr)

#     # Create data loaders
#     _bs = 32
#     train_loader = torch.utils.data.DataLoader(list(zip(X_train, y_train)), batch_size=_bs, shuffle=True)
#     test_loader = torch.utils.data.DataLoader(list(zip(X_test, y_test)), batch_size=_bs)
#     print(f"Batch size: {_bs}", f"lr: {_lr}")
#     # Training loop
#     num_epochs = 50
#     for epoch in range(num_epochs):
#         train_loss, train_accuracy = train_model(model, criterion, optimizer, train_loader, device)
#         test_loss, test_accuracy = test_model(model, criterion, test_loader, device)
#         print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")
#     test_loss, test_accuracy = test_model(model, criterion, test_loader, device)
#     print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

In [50]:
assert([file[:-3] for file in sorted(os.listdir(videos_folder))] == [file[:-3] for file in sorted(os.listdir(spectrograms_folder))])

In [51]:
X, y, skipped_files = load_dataset(videos_folder)
X_spec, y_spec = load_spectrogram_dataset(spectrograms_folder, skipped_files)

  0%|          | 1/7439 [00:15<32:32:45, 15.75s/it]

(115, 92, 3)


  0%|          | 3/7439 [00:18<8:52:58,  4.30s/it] 

(112, 89, 3)
(131, 101, 3)
(134, 101, 3)


  0%|          | 7/7439 [00:18<2:14:34,  1.09s/it]

...

100%|█████████▉| 7435/7439 [20:21<00:00, 10.39it/s]

(158, 127, 3)
(168, 132, 3)
(160, 128, 3)


100%|█████████▉| 7437/7439 [20:21<00:00, 11.38it/s]

(167, 131, 3)
(166, 132, 3)
(174, 137, 3)
(167, 131, 3)


100%|██████████| 7439/7439 [01:44<00:00, 70.91it/s]


In [52]:
# Save X, y, X_spec, y_spec
np.save('X.npy', np.array(X))
np.save('y.npy', np.array(y))

In [53]:
np.save('X_spec.npy', np.array(X_spec))
np.save('y_spec.npy', np.array(y_spec))

In [54]:
X_spec = np.load('/home1/riyaranj/riya/X_spec.npy', mmap_mode='r')
y_spec = np.load('/home1/riyaranj/riya/y_spec.npy', mmap_mode='r')
#X_spec_rgb = np.repeat(X_spec, 3, axis=1)  # Repeat the channels dimension
print(f"Total number of samples: {len(X_spec)}")
X_train, X_test, y_train, y_test = train_test_split(X_spec, y_spec, test_size=0.3, random_state=42)
print(f"Number of train samples: {len(X_train)}", f"Number of test samples: {len(X_test)}")

Total number of samples: 7429
Number of train samples: 5200 Number of test samples: 2229


In [56]:
if __name__ == "__main__":
    # Load dataset and split into train and test sets
   
    # Hyperparameters
    num_epochs = 50  # Example values, adjust as needed
    batch_sizes = [16, 32, 64]  # Example values, adjust as needed
    learning_rates = [0.01, 0.001, 0.0001]

    best_model_info = {}
    results_file = 'Latest_GoogLeNet_melspec_to_features_fullscale.csv'

    with open(results_file, mode='w', newline='') as csvfile:
        fieldnames = ['Model', 'Batch Size', 'Learning Rate', 'Num Epochs',  'Train Loss', 'Train Accuracy', 'Test Loss', 'Test Accuracy']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for lr in learning_rates:
            for bs in batch_sizes:
                # Initialize the model
                model = CNN(num_classes=6)  
                device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
                model.to(device)

                # Define loss function and optimizer
                criterion = nn.CrossEntropyLoss()
                optimizer = optim.Adam(model.parameters(), lr=lr)

                # Create data loaders
                train_loader = torch.utils.data.DataLoader(list(zip(X_train, y_train)), batch_size=bs, shuffle=True)
                test_loader = torch.utils.data.DataLoader(list(zip(X_test, y_test)), batch_size=bs)
                print(f"Batch size: {bs}", f"Num of Epochs: {num_epochs}", f"lr: {lr}")
                # Training loop
                for epoch in range(num_epochs):
                        print("Epoch " + str(epoch))
                        train_loss, train_accuracy = train_model(model, criterion, optimizer, train_loader, device)
                  # Calculate test loss and accuracy
                        test_loss, test_accuracy = test_model(model, criterion, test_loader, device)
                        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

                # Write results to CSV
                model_info = {'Model': f'Model_lr{lr}_bs{bs}_epochs{num_epochs}', 'Batch Size': bs, 'Learning Rate': lr, 'Num Epochs': num_epochs,
                              'Train Loss': train_loss, 'Train Accuracy': train_accuracy,
                              'Test Loss': test_loss, 'Test Accuracy': test_accuracy}
                writer.writerow(model_info)

                model_name = f'Latest_GoogLeNet_melspec_to_features_fullscale{num_epochs}_{bs}_{lr}'
                torch.save(model.state_dict(), model_name)

                # Check if this model is the best so far
                if not best_model_info or test_accuracy > best_model_info['Test Accuracy']:
                    best_model_info = {'Model': model_info['Model'], 'Batch Size': bs, 'Learning Rate': lr,
                                       'Train Loss': train_loss, 'Train Accuracy': train_accuracy,
                                       'Test Loss': test_loss, 'Test Accuracy': test_accuracy}

    # Print best model
    print("\nBest Model:")
    for key, value in best_model_info.items():
        print(f"{key}: {value}")


/home1/riyaranj/.local/lib/python3.12/site-packages/torchvision/models/googlenet.py:47: FutureWarning: The default weight initialization of GoogleNet will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


Batch size: 16 Num of Epochs: 50 lr: 0.01
Epoch 0


100%|██████████| 140/140 [00:02<00:00, 68.07it/s]


Epoch 1/50, Train Loss: 1.7863, Train Accuracy: 0.2535, Test Loss: 1.7307, Test Accuracy: 0.3122
Epoch 1


100%|██████████| 140/140 [00:02<00:00, 67.15it/s]


...

Epoch 48/50, Train Loss: 1.1222, Train Accuracy: 0.9244, Test Loss: 1.4341, Test Accuracy: 0.6039
Epoch 48


100%|██████████| 35/35 [00:01<00:00, 19.73it/s]


Epoch 49/50, Train Loss: 1.1107, Train Accuracy: 0.9367, Test Loss: 1.4340, Test Accuracy: 0.6039
Epoch 49


100%|██████████| 35/35 [00:01<00:00, 19.69it/s]


Epoch 50/50, Train Loss: 1.1124, Train Accuracy: 0.9344, Test Loss: 1.4153, Test Accuracy: 0.6245

Best Model:
Model: Model_lr0.0001_bs64_epochs50
Batch Size: 64
Learning Rate: 0.0001
Train Loss: 1.1124346839464627
Train Accuracy: 0.9344230769230769
Test Loss: 1.4152503232883198
Test Accuracy: 0.6244952893674294


In [57]:
print("Fin")

Fin
